# MyFariy AI 프롬프트
## 기능 명세
1. 낱말로부터 동화 생성
2. 음성으로 낱말 인식
3. 그림에서부터 동화 삽화 생성
4. 동화 내용 물어보며 대화하기

In [2]:
import app.config.init_config
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# 동화에서 키워드 문장 추출
---
# Task Description

Create a process that takes the title of a fairy tale as input and identifies four words from the story, excluding the characters, that can be replaced for adaptation. Each word must be replaceable by "XX," forming a total of four sentences, one for each word. The chosen words should represent objects or concepts that can be visually depicted.

---

## Steps

1. Understand the plot of the fairy tale based on the given title.
2. Identify four key words from the story that can be visualized and are not the names of characters. Each word should refer to a distinct object or concept.
3. Replace each selected word with "XX" in a sentence and construct four sentences.
4. translate the sentences into Korean.
---

### Output Format

- The output consists of four sentences, each with one word replaced by "XX."
- Example: "Once upon a time, there was a kingdom called XX."

---

### Examples

**Input:** The Three Little Pigs  
**Output:**
1. The first little pig built a house made of XX.  
2. The second little pig built a house made of XX.  
3. The third little pig built a house made of XX.  
4. The wolf destroyed the little pigs’ house with XX.  

---

### Notes

- Do not select character names as replaceable words.  
- Ensure the sentences are natural and align with the plot of the story.  
- Use descriptions that stay true to the scenes or narrative of the fairy tale.  

In [5]:
from app.core.chains.extract_sentence_from_title import prompt as extract_sentence_from_title_prompt, Parser as ExtractSentenceFromTitleOutputParser
chain = extract_sentence_from_title_prompt | llm | ExtractSentenceFromTitleOutputParser()

response = chain.invoke({"question": "성냥팔이 소녀"})
print(response)

['성냥팔이 소녀는 XX을 피워 따뜻함을 느꼈습니다.', '성냥팔이 소녀는 XX을 보며 행복한 꿈을 꾸었습니다.', '성냥팔이 소녀는 XX을 따라 떠나 행복한 세상을 상상했습니다.', '성냥팔이 소녀는 XX을 통해 가족과 함께 행복한 시간을 보냈습니다.']


# 문장 4개로 동화 생성
---
당신은 동화책 각색 작가입니다. 아래 제공된 각색된 동화 내용을 참고하여, 해당 내용을 창의적으로 계속 이어나가십시오.

# Requirements

- 필수문장을 반드시 포함해야 합니다.
- 이야기는 4개의 문단으로 이루어져야 합니다.
- 각 문단은 200글자 내외로 작성되어야 합니다.

# Steps

1. 기존 각색 내용을 기반으로 스토리를 확장하십시오.
2. 필수문장을 각 문단에 자연스럽게 통합하십시오.
3. 문단을 구분하여 각 200글자 내외로 서술하십시오.

# Output Format

이야기는 4개의 문단 형식으로, 각 문단은 약 200글자로 구성되도록 작성해 주십시오.

# Notes

- 필수문장을 매우 자연스럽게 통합하여 이야기를 방해하지 않도록 구성하십시오.
- 각 문단을 분리하여 표시하시고, 내용의 흐름이 논리적이고 매끄럽게 이어지도록 하십시오.

In [3]:
from app.core.chains.create_tale import prompt as create_tale_prompt, Parser as CreateTaleOutputParser
chain = create_tale_prompt | ChatOpenAI(temperature=0.1,model="gpt-4o") | CreateTaleOutputParser()

response = chain.invoke({
    "title": "아기돼지 삼형제",
    "introduction": "옛날 옛적에 아기돼지 삼형제가 살고 있었습니다. 이들은 각자 자신만의 집을 짓기로 결정했어요. 늑대는 첫째 돼지의 집에 와서 말했어요. \"문을 열어라!\"",
    "sentences": ['첫번째 아기돼지는 구름으로 만든 집을 지었습니다.', '두번째 아기돼지는 깃털으로 만든 집을 지었습니다.', '세번째 아기돼지는 초콜릿으로 만든 집을 지었습니다.', '늑대는 아기돼지의 집을 망치로 무너뜨렸습니다.'] 
})

print("\n".join(text for text in response))

"구름은 가볍고 부드러우니 집을 짓기에 좋을 거야!" 그는 하늘에서 구름을 모아 집을 지었습니다. 구름으로 만든 집은 마치 꿈속의 성처럼 아름다웠습니다. 하지만 늑대가 찾아와 "문을 열어라!"라고 외쳤을 때, 첫째 돼지는 두려움에 떨었습니다. 늑대가 숨을 크게 들이마시고 불자, 구름으로 만든 집은 순식간에 사라져 버렸습니다. 첫째 돼지는 형제들에게 달려가며 도움을 청했습니다. "형들, 늑대가 내 집을 날려버렸어!"
"깃털은 가볍지만 서로 엮으면 튼튼할 거야!" 그는 새들의 깃털을 모아 아늑한 집을 만들었습니다. 그러나 늑대가 다시 찾아와 "문을 열어라!"라고 외쳤습니다. 두번째 돼지는 겁에 질렸지만, 깃털 집이 버텨주길 바랐습니다. 하지만 늑대가 또다시 강하게 불을 내뿜자, 깃털로 만든 집은 허공으로 흩어졌습니다. 두번째 돼지도 형제들에게 달려가며 외쳤습니다. "늑대가 내 집도 날려버렸어!"
"초콜릿은 단단하고 맛있으니까 늑대도 쉽게 부수지 못할 거야!" 그는 달콤한 초콜릿 벽돌로 튼튼한 집을 지었습니다. 늑대가 찾아와 "문을 열어라!"라고 외쳤지만, 세번째 돼지는 자신만만했습니다. 그러나 늑대는 이번엔 숨을 불지 않고, 아기돼지의 집을 망치로 무너뜨렸습니다. 초콜릿 조각들이 사방으로 흩어졌고, 세번째 돼지도 형제들과 함께 도망쳐야 했습니다. "우린 이제 어떻게 해야 하지?"
이번에는 벽돌과 시멘트를 사용해 단단한 집을 지었습니다. 늑대가 다시 찾아와 "문을 열어라!"라고 외쳤지만, 삼형제는 두려워하지 않았습니다. 늑대는 숨을 불어보기도 하고 망치를 휘둘러보기도 했지만, 집은 끄떡없었습니다. 결국 늑대는 포기하고 떠났고, 삼형제는 안전하게 행복하게 살 수 있었습니다. "함께라면 무엇이든 이겨낼 수 있어!" 그들은 서로를 믿으며 웃음을 지었습니다.


In [4]:
scenes = response

# 이미지 프롬프트 생성
---
Create a set of prompts for an AI illustration tool intended to generate images for a children's storybook. You will receive the storybook title and four paragraphs, each describing a scene to illustrate. Your task is to use the information from the title and each paragraph to craft a specific prompt for the Stable Diffusion model, as well as a negative prompt that specifies what should not be included.

# Steps

1. **Analyze Inputs**: Carefully read the storybook title and each paragraph to identify key details, themes, characters, settings, and emotions.
2. **Create Stable Diffusion Prompts**:
   - For each paragraph, use the details identified to draft a clear and imaginative prompt that vividly describes the desired illustration.
   - Ensure the prompt captures the mood and style suitable for a children's storybook.
3. **Draft Negative Prompts**:
   - Highlight elements that should be excluded from each illustration, such as inappropriate themes, anachronisms, or specific unwanted details.
4. **Repeat for all Paragraphs**: Complete the process for each of the four paragraphs provided.

# Output Format

Provide each set of prompts as follows:
- **Prompt**: Write a descriptive sentence or paragraph for each input paragraph that outlines the desired illustration.
- **Negative Prompt**: Specify a list of elements or themes that should not appear in the illustration.

# Examples

Example Input:

Title: 아기돼지 삼형제
Scene
['늑대가 첫째 돼지의 깃털 집을 후~ 하고 불어 날려버렸어요. 첫째 돼지는 놀라 도망쳐 둘째 돼지의 나뭇가지 집으로 갔습니다.','둘째 돼지는 숲에서 깃털로 집을 짓기 시작했어요. 집은 무지개처럼 빛났지만 형제들은 약하다고 걱정했습니다.','셋째 돼지는 초콜릿 블록으로 튼튼한 집을 짓고, 사탕으로 장식했어요. 형제들은 모두 초콜릿 집의 튼튼함에 감탄했습니다.','늑대는 첫째, 둘째 돼지의 집을 날려버렸지만 초콜릿 집은 부수지 못했습니다. 형제들은 초콜릿 집에서 안전하게 살았습니다.']

Output
1.
Prompt: A dramatic moment where a large wolf is blowing down a delicate feather house in a lush forest. The feathers scatter like snow in the wind, while the frightened pig runs away toward another house in the distance. The mood is tense but still whimsical and lighthearted.
Negative Prompt: No dark tones, harsh lighting, menacing or violent imagery, overly realistic textures, or stormy weather.
2.
Prompt: A cheerful pig proudly building a colorful feather house in a sunny forest clearing. The feathers shimmer like a rainbow, creating a magical and delicate look. The pig appears confident and happy, surrounded by skeptical siblings.
Negative Prompt: No dark colors, messy backgrounds, or overly realistic or industrial designs.
3.
Prompt: A cozy, inviting house made of glossy chocolate blocks, decorated with colorful candies. The pig is happily putting on the final touches while the warm sunlight enhances the chocolate's shine. Nearby, the other pigs look on with admiration.
Negative Prompt: No dull colors, chaotic details, or cold and lifeless settings.
4.
Prompt: A tense but whimsical scene of the wolf trying to blow down or hammer the chocolate house, which stands firm and unyielding. The three pigs peek out of the safe house, looking relieved and united. The sunny forest adds warmth to the triumphant mood.
Negative Prompt: No overly scary wolf imagery, dark or gloomy atmosphere, or lack of natural forest elements.


# Notes

- Ensure each prompt is positive and suitable for a children's audience.
- Pay attention to cultural, seasonal, or historical elements indicated in the story title or paragraphs.

In [6]:
from app.core.chains.generate_image_prompt import prompt as generate_image_prompt, Parser as GenerateImageOutputParser
chain = generate_image_prompt | llm | GenerateImageOutputParser()

response = chain.invoke({"title":"아기돼지 삼형제", "scenes": scenes})
print(response)

[{'Prompt': 'A cute little pig building a house made of fluffy clouds in a bright and dreamy sky. The pig looks joyful and imaginative, surrounded by the softness of the clouds. The scene radiates a sense of lightness and creativity.', 'Negative Prompt': 'No dark or stormy clouds, sharp objects, or scary expressions on the characters.'}, {'Prompt': "An adorable pig constructing a cozy house out of colorful feathers in a whimsical meadow. The feathers create a soft and vibrant shelter, reflecting the pig's creativity and playfulness. The atmosphere is light-hearted and enchanting.", 'Negative Prompt': 'No sharp feathers, gloomy backgrounds, or overly intricate details that may overwhelm the scene.'}, {'Prompt': 'A delightful pig putting together a sturdy house made of rich chocolate in a sunny chocolate land. The house is adorned with sweet candies, exuding a warm and inviting feel. The pig appears content and proud of the chocolate creation.', 'Negative Prompt': 'No messy chocolate tex

# 동화 읽기
---
입력으로 주어진 내용은 절대 수정하지 않고 그대로 출력합니다. 

어린이 동화를 연기하며 읽는 모델을 구현하세요. 
동화 스크립트를 수정하지 않고 있는 글을 그대로 출력하세요.
동화의 대사 부분에서는 연기 톤으로 대사를 읽고, 그 외의 부분은 몰입감 있는 동화 내용을 전달하세요.

# Steps

1. **이해하기**: 먼저 동화 속 상황과 캐릭터들을 분석합니다. 각 캐릭터의 성격과 감정적인 배경을 이해하는 것이 중요합니다.
   
2. **분석 및 분배**: 이야기에 나오는 대사를 각 캐릭터의 성격에 맞게 분배합니다. 각각의 캐릭터가 가지고 있는 주요 특성을 반영하여 대사를 어떻게 전달할지 계획합니다.

3. **연기 적용**: 각 캐릭터의 대사를 연기하며 전달합니다. 목소리 톤, 감정 표현, 발음 등을 고려하여 각 대사를 효율적으로 표현합니다.
   
4. **내용 전달**: 대사가 아닌 부분은 이야기의 몰입감을 높이는 방식으로 전체적인 내용을 전달합니다.

# Notes

- 주어진 내용을 수정하지 않습니다.
- 캐릭터별 감정과 목소리의 일관성을 유지하는 것이 중요합니다.
- 대사의 전달이 내용의 이해를 돕도록 신경을 씁니다.

In [11]:
from openai import OpenAI
import base64
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o-audio-preview",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "text": """
입력으로 주어진 내용은 절대 수정하지 않고 그대로 출력합니다. 

어린이 동화를 연기하며 읽는 모델을 구현하세요. 
동화 스크립트를 수정하지 않고 있는 글을 그대로 출력하세요.
동화의 대사 부분에서는 연기 톤으로 대사를 읽고, 그 외의 부분은 몰입감 있는 동화 내용을 전달하세요.

# Steps

1. **이해하기**: 먼저 동화 속 상황과 캐릭터들을 분석합니다. 각 캐릭터의 성격과 감정적인 배경을 이해하는 것이 중요합니다.
   
2. **분석 및 분배**: 이야기에 나오는 대사를 각 캐릭터의 성격에 맞게 분배합니다. 각각의 캐릭터가 가지고 있는 주요 특성을 반영하여 대사를 어떻게 전달할지 계획합니다.

3. **연기 적용**: 각 캐릭터의 대사를 연기하며 전달합니다. 목소리 톤, 감정 표현, 발음 등을 고려하여 각 대사를 표현합니다.
   
4. **내용 전달**: 대사가 아닌 부분은 이야기의 몰입감을 높이는 방식으로 내용을 전달합니다.

# Notes

- 주어진 내용을 수정하지 않습니다.
- 캐릭터별 감정과 목소리의 일관성을 유지하는 것이 중요합니다.
- 대사의 전달이 내용의 이해를 돕도록 신경을 씁니다.""",
          "type": "text"
        }
      ]
    },
    {
        "role": "user",
        "content": [
            {
              "text":'"구름은 가볍고 부드러우니 집을 짓기에 좋을 거야!" 그는 하늘에서 구름을 모아 집을 지었습니다. 구름으로 만든 집은 마치 꿈속의 성처럼 아름다웠습니다. 하지만 늑대가 찾아와 "문을 열어라!"라고 외쳤을 때, 첫째 돼지는 두려움에 떨었습니다. 늑대가 숨을 크게 들이마시고 불자, 구름으로 만든 집은 순식간에 사라져 버렸습니다. 첫째 돼지는 형제들에게 달려가며 도움을 청했습니다. "형들, 늑대가 내 집을 날려버렸어!"',
              "type":"text"
            },
          ],
    },
  ],
  modalities=["text", "audio"],
  audio={
    "voice": "coral",
    "format": "wav"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

wav_bytes = base64.b64decode(response.choices[0].message.audio.data)
with open("dog.wav", "wb") as f:
    f.write(wav_bytes)